In [1]:
from pyspark.sql import SparkSession
# from pyspark.sql.types import DoubleType, FloatType, LongType, StructType,StructField, StringType

In [2]:
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

25/08/11 11:45:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
spark

In [5]:
spark.sql("CREATE NAMESPACE retail.raw_input")

DataFrame[]

In [6]:
spark.sql("SHOW NAMESPACES IN retail").show()

+---------+
|namespace|
+---------+
|raw_input|
+---------+



In [7]:
spark.sql("USE NAMESPACE raw_input")

DataFrame[]

In [8]:
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------+---------+
|catalog|namespace|
+-------+---------+
| retail|raw_input|
+-------+---------+



In [ ]:
# spark.sql("DROP TABLE IF EXISTS retail.customers")

25/08/11 11:46:51 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/08/11 11:46:51 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/08/11 11:47:01 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/08/11 11:47:01 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.18.0.8
25/08/11 11:47:01 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
25/08/11 11:47:02 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


DataFrame[]

In [10]:
df = spark.read.csv("/home/iceberg/data/customers.csv", header=True)

In [11]:
df.toPandas().head(5)

,customer_id,customer_address,customer_state,customer_zip_code_prefix,customer_created_date,customer_updated_date
0,00000001,1796 Martin Squares Apt. 839,IA,51083,2023-03-22,None
1,00000002,07396 Smith Ridge,VT,05422,2020-09-03,None
2,00000003,7698 Randall Vista Suite 922,MA,01047,2021-05-29,None
3,00000004,2426 Graham Mount Suite 333,MD,21016,2020-02-09,None
4,00000005,5992 Julie Oval,PW,96940,2020-01-16,None


In [18]:
spark.stop()

In [12]:
df.write.saveAsTable("customers")

In [13]:
spark.sql("SELECT * FROM customers LIMIT 5").show()

+-----------+--------------------+--------------+------------------------+---------------------+---------------------+
|customer_id|    customer_address|customer_state|customer_zip_code_prefix|customer_created_date|customer_updated_date|
+-----------+--------------------+--------------+------------------------+---------------------+---------------------+
|   00000001|1796 Martin Squar...|            IA|                   51083|           2023-03-22|                 NULL|
|   00000002|   07396 Smith Ridge|            VT|                   05422|           2020-09-03|                 NULL|
|   00000003|7698 Randall Vist...|            MA|                   01047|           2021-05-29|                 NULL|
|   00000004|2426 Graham Mount...|            MD|                   21016|           2020-02-09|                 NULL|
|   00000005|     5992 Julie Oval|            PW|                   96940|           2020-01-16|                 NULL|
+-----------+--------------------+--------------

In [14]:
spark.sql("UPDATE customers SET customer_updated_date='2025-08-11' WHERE customer_id=00000001")

DataFrame[]

In [15]:
spark.stop()